Goal: 각 item의 meta 정보를 연결해서 document로 만들고 이를 db로 저장

In [1]:
import os
import argparse
config = dict()
config['langchain_key'] = '/home/doyooni303/API_KEYS/langchain.txt'
config['openai_key'] = '/home/doyooni303/API_KEYS/OpenAI.txt'


def set_environ(config):
    os.environ['LANGCHAIN_TRACING_V2'] = 'true'
    os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
    with open(config['langchain_key'],'r') as f: langchain_key = f.readline()
    os.environ['LANGCHAIN_API_KEY'] = langchain_key
    with open(config["openai_key"],'r') as f: openai_key = f.readline()
    os.environ['OPENAI_API_KEY'] = openai_key

set_environ(config)


In [2]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
# item meta data 불러오기
import json
config['DATASET']=dict()
config['DATASET']['name'] = 'Books'
config['DATASET']['folder'] = '/home/doyooni303/experiments/sequential_recsys/datasets/amazon'
item2idx = json.load(open(os.path.join(config['DATASET']['folder'],
                                       config['DATASET']['name'],
                                       f"{config['DATASET']['name']}_item2idx.json"),"r"))

In [4]:
type(list(item2idx.values())[0])

int

In [5]:
itemidx2meta = json.load(open(os.path.join(config['DATASET']['folder'],
                                       config['DATASET']['name'],
                                       f"{config['DATASET']['name']}_itemidx2meta.json"),"r"))

In [7]:
itemidx2meta = {int(key):value for key, value in itemidx2meta.items()}

In [9]:
meta = itemidx2meta[0]
meta_keys = list(meta.keys())
meta,meta_keys

{'main_category': 'Books',
 'title': 'Chaucer',
 'subtitle': 'Hardcover – Import, January 1, 2004',
 'author': {'avatar': 'https://m.media-amazon.com/images/I/21Je2zja9pL._SY600_.jpg',
  'name': 'Peter Ackroyd',
  'about': ['Peter Ackroyd, (born 5 October 1949) is an English biographer, novelist and critic with a particular interest in the history and culture of London. For his novels about English history and culture and his biographies of, among others, William Blake, Charles Dickens, T. S. Eliot and Sir Thomas More, he won the Somerset Maugham Award and two Whitbread Awards. He is noted for the volume of work he has produced, the range of styles therein, his skill at assuming different voices and the depth of his research.',
   'He was elected a fellow of the Royal Society of Literature in 1984 and appointed a Commander of the Order of the British Empire in 2003.',
   'Bio from Wikipedia, the free encyclopedia.']},
 'average_rating': 4.5,
 'rating_number': 29,
 'features': [],
 'des

In [ ]:
# documents 만들기
from langchain_core.prompts import PromptTemplate
class Prompt:
    def __init__(self,meta:dict, *args) -> None:
        self.meta = meta
        self.args = args

In [ ]:
# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Embed
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

#### RETRIEVAL and GENERATION ####

# Prompt
prompt = hub.pull("rlm/rag-prompt")

# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Question
answer = rag_chain.invoke("What is Task Decomposition?")